In [12]:
import numpy as np
import math
from numpy import *

In [13]:
#algoritmo para predecir la posicion en el espacio

In [14]:
#algoritmo de Newtom Rhapson
#f,df son la funcion y su derivada
#Vin y N son el estimativo inicial y el numero de iteraciones 
def nr(f,df,Vin,N):
    p=Vin
    for i in xrange(0,N):
        p=p-(f(p)/df(p))
    return p

In [15]:
#Ecuacion de Kepler
#los valores de M,E se dan en radianes
def eckepler(E):
    kepler=E-e*np.sin(E)-M
    return kepler
#derivada ecuacion de kepler
def deckepler(E):
    dkepler=1-e*np.cos(E)
    return dkepler

In [16]:
#matris de rotacion, angulos en radianes
def rotacion(omega,inc,w):
    rot1=np.array([[np.cos(omega),-np.sin(omega),0.0],
             [np.sin(omega),np.cos(omega),0.0],
             [0.0,0.0,1.0]])
    rot2=np.array([[1.0,0.0,0.0],
               [0.0,np.cos(inc),-np.sin(inc)],
               [0.0,np.sin(inc),np.cos(inc)]])
    rot3=np.array([[np.cos(w),-np.sin(w),0.0],
             [np.sin(w),np.cos(w),0.0],
             [0.0,0.0,1.0]])
    rot_final=(rot1.dot(rot2)).dot(rot3)
    return rot_final

In [17]:
#anomalia verdadera, E en grados
def anomalia_verdadera(E,e):
    factor=((1.0+e)/(1.0-e))**0.5
    f=2.0*np.arctan(factor*np.tan(E/2.0))
    return f

In [18]:
#datos

In [19]:
grad=180.0/np.pi
rad=np.pi/180.0

In [20]:
#unidades canónicas
um=1.988544*10**30.0
ul=149597870700.0
G=6.67392*10**(-11.0)
ut=(ul**3.0/(um*G))**0.5
mu=(G*um)*(ut**2.0/ul**3.0)
dia=ut*(1.0/3600.0)*(1.0/23.9345)
print ut,dia,mu

5022616.48057 58.2912216509 1.0


In [21]:
tp=2457427.543855121499    #fecha juliana, dias,2016-Feb-09.04385512
t=2458459.2916667          #fecha juliana, dias UT-5 2018/02/06 14:00:00
#t=2458459.0833333          #fecha juliana, dias UT    2018/02/06 14:00:00
#t=2457430.0833333          #2016-feb-11 14:00:00
periodo=2047.343285965675  #dias
e=0.1872699504244361       #ecentricidad
a=3.155451391965179        #semieje mayor
#n=2*np.pi/periodo          #mean orbital motion numericamente rad/dia
n=0.1758376342979521*rad   #valor de Horizons rad/dia
M=n*(t-tp)                 #anomalia media  
print 'M =',M*grad

M = 181.42009438


In [22]:
#anomalia ecentrica y verdadera
E=nr(eckepler,deckepler,M,10)
#E=Esol(M,e)
f1=1.0*anomalia_verdadera(E,e)
if f1<0.0:
    f=2*np.pi+f1
cero=E-e*np.sin(E)-M #verificacion de E, debe dar cero esta linea
print 'E =',E*grad
print 'f =',f*grad
print 'Aqui debe dar cero',cero

E = 181.196114371
f = 180.989637412
Aqui debe dar cero 0.0


In [32]:
#posicion y velosidad en el plano de referencia
p=a*(1.0-e**2)                                         #semilatus rectum
v1=-1.0*(mu/p)**0.5                                    #factor para v
r_mag=p/(1.0+e*np.cos(f))                              #magnitud de r con f
#r_mag=a*(1-e*np.cos(E))                                #magnitud de r con E
r=np.array([r_mag*np.cos(f),r_mag*np.sin(f),0.0])      #r vector
vel=np.array([v1*np.sin(f),-1.0*v1*(e+np.cos(f)),0.0]) #v vector
print 'r =',r
print 'v =',vel

r = [-3.74568505 -0.06470352  0.        ]
v = [ 0.00989813 -0.4656804   0.        ]


In [33]:
#elementos orbitales
#tomado de Horizons
omega=110.9390420742303*rad
w=358.7790635893293*rad
inc=2.73824776304453*rad

In [34]:
#matris de rotacion
rot_final=rotacion(omega,inc,w)

In [35]:
r_rot=rot_final.dot(r)
v_rot=rot_final.dot(vel)
print 'r rotado =',r_rot
print 'v rotada =',(1.0/dia)*v_rot  #de ul por ut a ul por dia

r rotado = [  1.32469267e+00  -3.50421630e+00   7.22488201e-04]
v rotada = [ 0.00745461  0.00285197 -0.00038174]


In [36]:
""""
datos de Horizons:
2458459.083333333 = A.D. 2018-Dec-06 14:00:00.0000 (TDB)
   1.308057620446669E+00 -3.510316245875934E+00  1.194654235809810E-03
   7.457989109089521E-03  2.817550176085307E-03 -3.817166437810427E-04
   2.163572403152746E-02  3.746109490080055E+00 -3.616250114796165E-05
   
2457430.083333333 = A.D. 2016-Feb-11 14:00:00.0000 (TDB)
  -8.894479089857996E-01  2.405670048896956E+00 -1.352903896373736E-03
  -1.096617715140197E-02 -4.044752792298253E-03  5.591054548037377E-04
   1.481323942244966E-02  2.564832898515621E+00  8.868520566378756E-06
"""

'"\ndatos de Horizons:\n2458459.083333333 = A.D. 2018-Dec-06 14:00:00.0000 (TDB)\n   1.308057620446669E+00 -3.510316245875934E+00  1.194654235809810E-03\n   7.457989109089521E-03  2.817550176085307E-03 -3.817166437810427E-04\n   2.163572403152746E-02  3.746109490080055E+00 -3.616250114796165E-05\n   \n2457430.083333333 = A.D. 2016-Feb-11 14:00:00.0000 (TDB)\n  -8.894479089857996E-01  2.405670048896956E+00 -1.352903896373736E-03\n  -1.096617715140197E-02 -4.044752792298253E-03  5.591054548037377E-04\n   1.481323942244966E-02  2.564832898515621E+00  8.868520566378756E-06\n'